Сбор данных проведен с помощью файлов Parser и Parser_reviews. Кроме того проведена предобработка данных в файле Preprocessing.

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv('preprocessed_data.csv')

In [3]:
data = data.drop([
    col for col in data.columns if col not in ['Reviews', 'Score']
], axis=1)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5255 entries, 0 to 5254
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Score    5255 non-null   float64
 1   Reviews  5255 non-null   object 
dtypes: float64(1), object(1)
memory usage: 82.2+ KB


In [5]:
data.describe()

,Score
count,5255.000000
mean,8.431066
std,0.679967
min,3.440000
25%,8.000000
50%,8.540000
75%,8.970000
max,10.000000


In [6]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
def report(test_y, preds):
    print("Mean squared error: %.3f" % mean_squared_error(test_y, preds))
    print("Mean absolute error: %.3f" % mean_absolute_error(test_y, preds))
    print("Coefficient of determination: %.3f" % r2_score(test_y, preds))

# Токенизируем данные

In [7]:
import string
import nltk
from functools import lru_cache

In [8]:
stop_words = nltk.corpus.stopwords.words('russian')

In [9]:
from collections import namedtuple

@lru_cache(500000)
def new_func_01(data):
    word_tokenizer = nltk.WordPunctTokenizer()
    new_columns = []

    for index, item in tqdm(data):               
        text_lower = item.lower()
        tokens = word_tokenizer.tokenize(text_lower)        
        tokens = tuple([word for word in tokens 
                             if (word.isalpha() and word not in stop_words)])                             
        new_columns.append(tokens)    
    return tuple(new_columns)

def tokenize_data(columns):
    data = tuple(columns.items())  
    return new_func_01(data)

# Лемматизация

In [10]:
import pymorphy2

In [11]:
morph = pymorphy2.MorphAnalyzer()

In [12]:
@lru_cache(500000)
def lemmatize_data(columns):
    new_columns = pd.Series(dtype='object')
    new_item = ''
    for item in tqdm(columns):
        for word in item:
            new_word = morph.parse(word)[0].normal_form
            new_item = ' '.join([new_item, new_word])
        new_columns = pd.concat([new_columns, pd.Series(new_item)], ignore_index=True)
        new_item = ''
    return pd.DataFrame(new_columns, columns=['Reviews'])

## Подготовка данных

In [13]:
tokenized_reviews = tokenize_data(data['Reviews'])

  0%|          | 0/5255 [00:00<?, ?it/s]

In [14]:
lemmatized_reviews = lemmatize_data(tokenized_reviews)

  0%|          | 0/5255 [00:00<?, ?it/s]

In [15]:
data = data.drop(['Reviews'], axis=1)
data = pd.concat([data, lemmatized_reviews], axis=1)

In [16]:
from sklearn.model_selection import train_test_split
y = data['Score']
X_full = data
train_x, test_x, train_y, test_y = train_test_split(X_full, y, test_size=0.2, random_state=42)

## Основной pipeline для подбора оптимальных параметров TF-IDF

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Исключим этапы токенизации и лемматизации из основного pipeline, чтобы можно было использовать его для подбора оптимальных параметров TF-IDF.

In [18]:
reviews_params = dict(norm = None, max_features = 100, ngram_range=(1, 2), max_df=0.80, min_df=0.01)

In [19]:
ReviewsTextProcessor = Pipeline(steps=[
    ("squeez", FunctionTransformer(lambda x: x.squeeze())),
    ("tfidf", TfidfVectorizer(**reviews_params)),
    ("toarray", FunctionTransformer(lambda x: x.toarray()))
])

In [20]:
data_transformer = ColumnTransformer(
    transformers=[
        ("review", ReviewsTextProcessor, ['Reviews']),
    ])

In [21]:
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

In [22]:
regression_pipeline = Pipeline( 
    steps=[('preprocessor', preprocessor),
           ('model', LinearRegression())])

Получим некий начальный вариант для набора параметров.

In [23]:
regression_pipeline.fit(train_x, train_y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('review',
                                                                   Pipeline(steps=[('squeez',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x7f7e8c923c70>)),
                                                                                   ('tfidf',
                                                                                    TfidfVectorizer(max_df=0.8,
                                                                                                    max_features=100,
                                                                                                    min_df=0.01,
                                                                                                    ngram_range=(1,
                                                                                                                 2),
                                                                                                    norm=None)),
                                                                                   ('toarray',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x7f7e8c923d90>))]),
                                                                   ['Reviews'])]))])),
                ('model', LinearRegression())])

In [24]:
preds = regression_pipeline.predict(test_x)

In [25]:
report(test_y, preds)

Mean squared error: 0.386
Mean absolute error: 0.477
Coefficient of determination: 0.213


Используем этот pipeline для тестирования оптимальных значений для TfidfVectorizer и моделей. Подбор проведем в отдельном скрипте (optimizer.py). В результате получили следующие параметры:

In [26]:
def get_new_pipeline(vectorizer, model):
    """Функция для создания pipeline с указанным векторайзером и моделью
    """
    ReviewsTextProcessor = Pipeline(steps=[
        ("squeez", FunctionTransformer(lambda x: x.squeeze())),
        ("tfidf", vectorizer),
        ("toarray", FunctionTransformer(lambda x: x.toarray()))
    ])
        
    data_transformer = ColumnTransformer(
        transformers=[
            ("review", ReviewsTextProcessor, ['Reviews']),
    ])
        
    preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])
        
    pipeline = Pipeline( 
        steps=[('preprocessor', preprocessor),
            ('model', model)])
        
    return pipeline

In [27]:
def model_results(pipeline, train_x, test_x, train_y, test_y):
    """Универсальная функция для тестирвания модели и параметров. Передается только pipeline и данные.
    Выводит результаты модели функцией report.
    Возвращает DataFrame со словами из TF-IDF и соответствующими коэффициентами."""
    pipeline.fit(train_x, train_y)
    preds = pipeline.predict(test_x)
    
    vectorizer = pipeline['preprocessor']['data_transformer'].transformers_[0][1][1]
    words = pd.DataFrame(vectorizer.get_feature_names_out(), columns=['Слова'])
    values = pd.DataFrame(pipeline['model'].coef_, columns=['Коэффициенты'])
    results = pd.concat([words, values], axis=1)   
    results = results.sort_values(by='Коэффициенты', axis=0)
    report(test_y, preds)
    return results

In [28]:
reviews_params = dict(norm = None, max_features = 250, ngram_range=(1, 4), max_df=0.8, min_df=0.04)

vectorizer = TfidfVectorizer(**reviews_params)
model= LinearRegression()
optimal_regression_pipeline = get_new_pipeline(vectorizer, model)

results = model_results(optimal_regression_pipeline, train_x, test_x, train_y, test_y)

Mean squared error: 0.375
Mean absolute error: 0.469
Coefficient of determination: 0.235


# TF-IDF

Рассмортим влияние слов, выделенных TF-IDF, на целевую переменную.

In [29]:
from math import ceil
def table_transform(data, column_number):
    single_sift = ceil(data.shape[0] / column_number)
    pointer = 1
    new_data = data.copy()
    while pointer < column_number:
        new_data = pd.concat([new_data, data.shift(periods=-single_sift*pointer)], axis=1, ignore_index=True)
        pointer += 1
    return new_data[:single_sift]

Оценим список слов, внесших наибольший вклад в целевую переменную:

In [35]:
table_transform(results[abs(results['Коэффициенты']) >= 0.01], 6)

,0,1,2,3,4,5,6,7,8,9,10,11
160,полный,-0.034358,время,-0.014175,бесить,-0.010378,весь,0.011037,дыхание,0.014733,отличный,0.022174
183,рейтинг,-0.024383,понятно,-0.014017,дорама очень,-0.010225,дора,0.011089,мило,0.015533,крутой,0.025029
97,любитель,-0.020119,делать,-0.013140,классный дорама,-0.010127,сам,0.011221,пока,0.015634,однозначно,0.026292
221,сценарий,-0.018728,понять,-0.012841,супер,0.010064,советовать просмотр,0.011686,пересматривать,0.015980,милый,0.028023
155,плохой,-0.018504,таки,-0.012631,плакать,0.010112,главный,0.012122,надеяться,0.016326,хороший фильм,0.028130
25,главный героиня,-0.017990,оценка,-0.012571,удовольствие,0.010124,состав,0.012928,жанр,0.016794,классный,0.030400
114,найти,-0.017218,актриса,-0.012086,написать,0.010179,мир,0.013264,первый серия,0.017300,None,NaN
36,дело,-0.017130,сделать,-0.011781,стать,0.010394,шикарный,0.013568,химия,0.019474,None,NaN
15,впечатление,-0.016937,сериал,-0.011556,советовать,0.010414,прям,0.013944,рекомендовать,0.020486,None,NaN
148,перевод,-0.014692,перемотка,-0.011120,ожидать,0.010912,момент,0.013953,главное,0.021380,None,NaN


Видно, что результат достаточно объяснимый: слова 'плохой', 'бесить', 'перемотка' вносят существенный отрицательный вклад в рейтинг, а слова 'классный', 'милый', 'отличный', 'рекомендовать' - положительный. Хотя и итоговая эффективность модели достаточно низкая.

Попробуем подобрать оптимальные варианты с нормализацией. Начнем с Ridge.

In [32]:
from sklearn.linear_model import Ridge

reviews_params = dict(norm = None, max_features = 250, ngram_range=(1, 2), max_df=0.78, min_df=0.04)

vectorizer = TfidfVectorizer(**reviews_params)
model= Ridge(alpha=1)
ridge_pipeline = get_new_pipeline(vectorizer, model)

ridge_results = model_results(ridge_pipeline, train_x, test_x, train_y, test_y)

Mean squared error: 0.375
Mean absolute error: 0.469
Coefficient of determination: 0.235


In [36]:
table_transform(ridge_results[abs(ridge_results['Коэффициенты']) >= 0.01], 6)

,0,1,2,3,4,5,6,7,8,9,10,11
160,полный,-0.034355,время,-0.014175,бесить,-0.010377,весь,0.011036,дыхание,0.014699,отличный,0.022173
183,рейтинг,-0.024381,понятно,-0.014016,дорама очень,-0.010223,дора,0.011089,мило,0.015532,крутой,0.025026
97,любитель,-0.020117,делать,-0.013138,классный дорама,-0.010123,сам,0.011219,пока,0.015632,однозначно,0.026290
221,сценарий,-0.018726,понять,-0.012841,супер,0.010064,советовать просмотр,0.011684,пересматривать,0.015978,милый,0.028022
155,плохой,-0.018503,таки,-0.012629,плакать,0.010111,главный,0.012113,надеяться,0.016324,хороший фильм,0.028127
25,главный героиня,-0.017978,оценка,-0.012570,удовольствие,0.010123,состав,0.012924,жанр,0.016792,классный,0.030395
114,найти,-0.017216,актриса,-0.012084,написать,0.010177,мир,0.013263,первый серия,0.017296,None,NaN
36,дело,-0.017127,сделать,-0.011779,стать,0.010393,шикарный,0.013567,химия,0.019473,None,NaN
15,впечатление,-0.016935,сериал,-0.011556,советовать,0.010415,прям,0.013944,рекомендовать,0.020485,None,NaN
148,перевод,-0.014691,перемотка,-0.011118,ожидать,0.010910,момент,0.013952,главное,0.021376,None,NaN


Получили такое же значение ошибки. Список слов также очень похож.

Теперь L1 регуляризация.

In [37]:
from sklearn.linear_model import Lasso

reviews_params = dict(norm = None, max_features = 400, ngram_range=(1, 2), max_df=0.98, min_df=0.04)

vectorizer = TfidfVectorizer(**reviews_params)
model= Lasso(alpha=0.0135)
lasso_pipeline = get_new_pipeline(vectorizer, model)

lasso_results = model_results(lasso_pipeline, train_x, test_x, train_y, test_y)

Mean squared error: 0.365
Mean absolute error: 0.461
Coefficient of determination: 0.255


In [40]:
table_transform(lasso_results[abs(lasso_results['Коэффициенты']) >= 0.01], 6)

,0,1,2,3,4,5,6,7,8,9,10,11
251,полный,-0.022664,странный,-0.013282,круто,0.010453,давно,0.013489,отличный,0.018143,однозначно,0.023536
296,рейтинг,-0.021720,любитель,-0.012703,пересматривать,0.011004,думать,0.014179,рекомендовать,0.018353,милый,0.024632
11,бред,-0.019622,найти,-0.011694,момент,0.011282,трогательный,0.014207,высокий,0.018473,None,NaN
242,плохой,-0.016108,сериал,-0.010582,хороший,0.011625,рада,0.014366,прекрасно,0.019687,None,NaN
265,потратить,-0.015545,советовать,0.010015,надеяться,0.011877,восторг,0.014763,классный,0.020149,None,NaN
233,перевод,-0.014120,любимый,0.010251,прям,0.012553,химия,0.015359,хороший фильм,0.020257,None,NaN
352,сценарий,-0.013383,мило,0.010295,тяжёлый,0.012837,дыхание,0.015641,крутой,0.021699,None,NaN


Здесь метрики немного улучшены.
Также, как и следовало ожидать, Lasso отбросила незначительно влияющие слова. Теперь в списке практически все слова очевидны.

И, наконец, ElasticNet.

In [41]:
from sklearn.linear_model import ElasticNet

reviews_params = dict(norm = None, max_features = 850, ngram_range=(1, 3), max_df=0.90, min_df=0.03)

vectorizer = TfidfVectorizer(**reviews_params)
model= ElasticNet(alpha=2.2, l1_ratio=0.000001, random_state=42, max_iter=20000)
elasticnet_pipeline = get_new_pipeline(vectorizer, model)

elasticnet_results = model_results(elasticnet_pipeline, train_x, test_x, train_y, test_y)

Mean squared error: 0.352
Mean absolute error: 0.449
Coefficient of determination: 0.281


Примечательно, что при работе оптимизатора лучшие результаты были получены при коэффициенте L1 регуляризации практически равном нулю. Однако, при работе Ridge модели метрики были намного хуже. И эта модель оказалась более эффективна при большем числе слов для TF-IDF.

In [42]:
table_transform(elasticnet_results[abs(elasticnet_results['Коэффициенты']) >= 0.01], 6)

,0,1,2,3,4,5,6,7,8,9,10,11
390,низкий,-0.024609,появиться,-0.011002,тяжёлый,0.010503,отличный,0.011089,хороший фильм,0.012219,классный,0.013588
516,полный,-0.013931,плохой,-0.010453,крутой,0.010524,рекомендовать,0.011095,однозначно,0.012579,милый,0.015200
26,бред,-0.013108,фильм,0.010097,прекрасно,0.010786,шикарно,0.011698,топ,0.012824,бомба,0.015349


Список слов очень "чистый" и короткий. Хочется отметить, что впервые появилось слово 'бомба'. В более расширенном варианте:

In [43]:
table_transform(elasticnet_results[abs(elasticnet_results['Коэффициенты']) >= 0.005], 6)

,0,1,2,3,4,5,6,7,8,9,10,11
390,низкий,-0.024609,количество,-0.006608,посмотреть фильм,0.005047,главное герой,0.005643,дом,0.006764,рада,0.008936
516,полный,-0.013931,красота,-0.006577,часть,0.005108,избранный,0.005777,супер,0.006773,давно,0.008956
26,бред,-0.013108,голова,-0.006380,сам,0.005123,выйти,0.005785,это первый,0.006796,огонь,0.008984
559,появиться,-0.011002,любовь,-0.006235,красиво,0.005163,напряжение,0.005786,шедевр,0.006848,хороший,0.008988
495,плохой,-0.010453,мина,-0.006092,всё,0.005178,ниже,0.005809,пока,0.006930,химия,0.009386
614,рейтинг,-0.009710,сценарист,-0.006084,стать,0.005210,жанр,0.005892,юный,0.006953,трогательный,0.009498
731,странный,-0.009501,исторический,-0.006036,удовольствие,0.005276,затянуть,0.006000,весь,0.006962,вау,0.009557
781,ужасный,-0.009372,сделать,-0.006029,мой,0.005292,очень классный,0.006011,просмотр,0.007047,думать,0.009618
721,старый,-0.009283,понять,-0.006023,обожать,0.005299,захватывать,0.006085,дуже,0.007273,высокий,0.009830
360,найти,-0.009190,простой,-0.005985,плакать,0.005327,один дыхание,0.006122,весь советовать,0.007344,фильм,0.010097


Также следует отметить, что появилось много биграммов (прямое следствие расширения списка слов для TF-IDF). Однако, биграммы 'хороший фильм' и 'фильм хороший' это 2 независимые сущности.

В целом, эффективность модели не слишком высокая. Вероятно, это связано с некоторой хаотичностью исходных данных, (когда присутствуют и положительные, и отрицательные отзывы). Кроме того, не все люди, проставляющие рейтинги, оставляют отзывы.

# Проверка на полных данных

Проверим какую эффективность можно получить на всех собранных данных.

In [44]:
import pandas as pd
from tqdm.notebook import tqdm

In [45]:
full_data = pd.read_csv('preprocessed_data.csv')

In [46]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5255 entries, 0 to 5254
Data columns (total 40 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       5255 non-null   int64  
 1   Name             5255 non-null   object 
 2   Year             5255 non-null   int64  
 3   Translation      5255 non-null   float64
 4   Score            5255 non-null   float64
 5   Description      5255 non-null   object 
 6   Reviews          5255 non-null   object 
 7   genre_0          5255 non-null   int64  
 8   genre_1          5255 non-null   int64  
 9   genre_2          5255 non-null   int64  
 10  genre_3          5255 non-null   int64  
 11  genre_4          5255 non-null   int64  
 12  genre_5          5255 non-null   int64  
 13  genre_6          5255 non-null   int64  
 14  genre_7          5255 non-null   int64  
 15  genre_8          5255 non-null   int64  
 16  genre_9          5255 non-null   int64  
 17  genre_10      

In [47]:
full_data = full_data.drop('Unnamed: 0', axis=1)

In [48]:
stop_words = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

tokenized_reviews = tokenize_data(full_data['Reviews'])
lemmatized_reviews = lemmatize_data(tokenized_reviews)
full_data = full_data.drop(['Reviews'], axis=1)
full_data = pd.concat([full_data, lemmatized_reviews], axis=1)

tokenized_description = tokenize_data(full_data['Description'])
lemmatized_description = lemmatize_data(tokenized_description)
lemmatized_description.rename(columns={'Reviews': 'Description'}, inplace=True)

full_data = full_data.drop(['Description'], axis=1)
full_data = pd.concat([full_data, lemmatized_description], axis=1)

  0%|          | 0/5255 [00:00<?, ?it/s]

  0%|          | 0/5255 [00:00<?, ?it/s]

In [49]:
from sklearn.model_selection import train_test_split
new_y = full_data['Score']
new_X_full = full_data.drop('Name', axis=1)
new_train_x, new_test_x, new_train_y, new_test_y = train_test_split(
    new_X_full, new_y, test_size=0.2, random_state=42)

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


def get_full_pipeline():
    """Функция для создания полного pipeline с указанным векторайзером и моделью
    """
    review_feature = ['Reviews']

    description_feature = ['Description']

    numerical_features = [
        col for col in full_data.columns if col not in ['Reviews', 'Description', 'Name', 'Score']
    ]
    
    reviews_params = dict(norm = None, max_features = 200, ngram_range=(1, 3), max_df=0.80, min_df=0.02)
    description_params = dict(norm = None, max_features = 50, ngram_range=(1, 3), max_df=0.95, min_df=0.1)

    ReviewsTextProcessor = Pipeline(steps=[
        ("squeez", FunctionTransformer(lambda x: x.squeeze())),
        ("tfidf", TfidfVectorizer(**reviews_params)),
        ("toarray", FunctionTransformer(lambda x: x.toarray()))
    ])

    DescriptionTextProcessor = Pipeline(steps=[
        ("squeez", FunctionTransformer(lambda x: x.squeeze())),
        ("tfidf", TfidfVectorizer(**description_params)),
        ("toarray", FunctionTransformer(lambda x: x.toarray()))
    ])
        
    numerical_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),])
    
    data_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numerical_transformer, numerical_features),
        ("review", ReviewsTextProcessor, review_feature),
        ("description", DescriptionTextProcessor, description_feature)])
        
    preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])
        
    pipeline = Pipeline( 
        steps=[('preprocessor', preprocessor),
            ('model', ElasticNet(alpha=0.1, l1_ratio=0.000001, random_state=42, max_iter=10000))])
        
    return pipeline

Подбор параметров проведен в optimizer_for_full_data.py

In [52]:
full_pipeline = get_full_pipeline()

In [53]:
full_pipeline.fit(new_train_x, new_train_y)
prediction = full_pipeline.predict(new_test_x)
 
report(new_test_y, prediction)

Mean squared error: 0.320
Mean absolute error: 0.421
Coefficient of determination: 0.346


При использовании всех данных для построения модели, как и следовало ожидать, ошибка незначительно уменьшается.